In [ ]:
# Import all libraries at the top
import requests
from bs4 import BeautifulSoup
import smtplib
import time
import datetime
import csv

# Constants (settings we might tweak later)
URL = "https://www.sharesansar.com/company/ghl"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"
}
CSV_FILE = "NepseWebScrapperDataset.csv"

# Function to send email when price > 300
def send_email(price):
    sender = "nirpesh02@gmail.com"
    password = "App Password Here"  # Gmail App Password
    receivers = ["nirpesh02@gmail.com", "Shresthaprajum@gmail.com"]
    subject = "Price Alert: Over 350!"
    body = f"The price of GHL just hit {price} - it's above 350!"  # Plain quote for safety
    message = f"Subject: {subject}\n\n{body}"
    message = message.encode("utf-8")  # Handle special characters
    
    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(sender, password)
        server.sendmail(sender, receivers, message)
        server.quit()
        print(f"Email sent to {len(receivers)} people: Price is ${price}")
    except Exception as e:
        print(f"Email failed: {e}")

# Function to scrape and process data
def check_price():
    # Fetch webpage
    try:
        response = requests.get(URL, headers=HEADERS, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch page: Status code {response.status_code}")
            return
    except Exception as e:
        print(f"Error fetching page: {e}")
        return

    # Parse HTML
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Get and clean the title
    title_tag = soup.find("h1")
    if title_tag:
        raw_title = title_tag.text
        parts = raw_title.split("(", 1)
        cleaned_title = parts[0].strip() + "(" + parts[1].strip()
        print(f"Title: {cleaned_title}")
    else:
        cleaned_title = "Unknown Title"
        print("Title not found")

    # Get the price (fixed for sharesansar.com)
    price_row = soup.find("span", class_="comp-price")
    if price_row:
        #price_tag = price_row.find_all("td")[1]  # Second <td> has the price
        price = float(price_row.text.strip())
        print(f"Current Stock Price: ${price}")
    else:
        price = None
        print("Current Stock Price: Price not found")

    # Get today’s date
    today = datetime.date.today()

    # Save to CSV
    header = ["Title", "Price", "Date"]
    data = [cleaned_title, price, today]
    
    try:
        with open(CSV_FILE, "a+", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            # Check if file is empty to write header
            f.seek(0)
            if f.read(1) == "":
                writer.writerow(header)
            writer.writerow(data)
        print("Data appended to CSV")
    except Exception as e:
        print(f"CSV write failed: {e}")

    # Send email if price > 350
    if price and price > 350:
        send_email(price)
    else:
        print("Price is under $350 or not found - no email sent")

# Main loop
if __name__ == "__main__":
    print("Starting the price checker...")
    while True:
        check_price()
        print("Sleeping for 24 hours...")
        time.sleep(86400)  # 1 day

Starting the price checker...
Title: Ghalemdi Hydro Limited(GHL )
Current Stock Price: $312.0
Data appended to CSV
Price is under $350 or not found - no email sent
Sleeping for 24 hours...
